# 02 - GPU Operator Installation

Install NVIDIA GPU Operator in K3s cluster, confirm that Kubernetes can detect and schedule GPU resources
- [NVIDIA GPU Operator Docs](https://docs.nvidia.com/datacenter/cloud-native/gpu-operator/latest/index.html)

## Setup

In [2]:
import os

**Prerequisites**
- `nvidia-smi`
- `helm`
    ```bash
        curl https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3 | bash
    ```

## Install

### 1. Add NVIDIA Helm repo


Add the chart source required to install the GPU Operator

In [1]:
!helm repo add nvidia https://nvidia.github.io/gpu-operator

"nvidia" has been added to your repositories


In [2]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "nvidia" chart repository
Update Complete. ⎈Happy Helming!⎈


### 2. Install GPU Operator

Install the operator into its own namespace 

In [3]:
!helm install --wait gpu-operator nvidia/gpu-operator --create-namespace --namespace gpu-operator

I0803 22:11:52.207022  124609 warnings.go:110] "Warning: spec.template.spec.affinity.nodeAffinity.preferredDuringSchedulingIgnoredDuringExecution[0].preference.matchExpressions[0].key: node-role.kubernetes.io/master is use \"node-role.kubernetes.io/control-plane\" instead"
I0803 22:11:52.207267  124609 warnings.go:110] "Warning: spec.template.spec.affinity.nodeAffinity.preferredDuringSchedulingIgnoredDuringExecution[0].preference.matchExpressions[0].key: node-role.kubernetes.io/master is use \"node-role.kubernetes.io/control-plane\" instead"
NAME: gpu-operator
LAST DEPLOYED: Sun Aug  3 22:11:51 2025
NAMESPACE: gpu-operator
STATUS: deployed
REVISION: 1
TEST SUITE: None


### 3. Check operator pods  

Verify that the GPU operator components are running:

- `gpu-operator`: the controller that manages all GPU-related resources
- `nvidia-device-plugin-daemonset`: exposes GPUs to Kubernetes via `nvidia.com/gpu`
- `nvidia-dcgm-exporter`: collects GPU metrics for Prometheus
- `nvidia-container-toolkit-daemonset`: configures container runtimes for GPU access
- `gpu-feature-discovery`: labels the node with GPU capabilities
- `nvidia-operator-validator`: checks compatibility (drivers, kernel, etc.)

In [6]:
!kubectl get pods -n gpu-operator

NAME                                                          READY   STATUS     RESTARTS       AGE
gpu-feature-discovery-cqtwn                                   0/1     Init:0/1   0              6m10s
gpu-operator-5798b5b564-tll5t                                 1/1     Running    0              6m34s
gpu-operator-node-feature-discovery-gc-86f6495b55-mfbng       1/1     Running    0              6m34s
gpu-operator-node-feature-discovery-master-694467d5db-85dgr   1/1     Running    0              6m34s
gpu-operator-node-feature-discovery-worker-2rfmj              1/1     Running    0              6m34s
nvidia-container-toolkit-daemonset-xzhlc                      1/1     Running    5 (105s ago)   6m12s
nvidia-dcgm-exporter-ncjsn                                    0/1     Init:0/1   0              6m11s
nvidia-device-plugin-daemonset-t4vr8                          0/1     Init:0/1   0              6m11s
nvidia-operator-validator-zr9lc                               0/1     Init:0/4   0  

Run in the terminal:

```bash
    !curl -sfL https://get.k3s.io | sh -
```

Tell kubectl to use the kubeconfig generated by K3s

In [4]:
os.environ["KUBECONFIG"] = "/etc/rancher/k3s/k3s.yaml"

You may want to make the `/etc/rancher/k3s/k3s.yaml` file world-readable:
```bash
    sudo chmod 644 /etc/rancher/k3s/k3s.yaml
```

### Verify

In [6]:
!kubectl get nodes

NAME       STATUS   ROLES                  AGE     VERSION
laniakea   Ready    control-plane,master   8m10s   v1.33.3+k3s1


In [7]:
!kubectl get pods -A

NAMESPACE     NAME                                      READY   STATUS      RESTARTS   AGE
kube-system   coredns-5688667fd4-mmqbg                  1/1     Running     0          8m30s
kube-system   helm-install-traefik-crd-sm2vd            0/1     Completed   0          8m30s
kube-system   helm-install-traefik-vhzfv                0/1     Completed   1          8m30s
kube-system   local-path-provisioner-774c6665dc-tdw8s   1/1     Running     0          8m30s
kube-system   metrics-server-6f4c6675d5-ts7dz           1/1     Running     0          8m30s
kube-system   svclb-traefik-e02ee076-s64zm              2/2     Running     0          8m10s
kube-system   traefik-c98fdf6fb-nh2hh                   1/1     Running     0          8m10s


If the node is "Ready" and system pods are running - K3s cluster is healthy

## Test

Create a simple BusyBox pod that sleeps for 1 hour, to confirm the cluster can schedule workloads

In [9]:
%%writefile ../deployments/test-pod.yaml
apiVersion: v1
kind: Pod
metadata:
  name: k3s-test-pod
spec:
  containers:
  - name: busybox
    image: busybox
    command: ["sleep", "3600"]

Writing ../deployments/test-pod.yaml


In [11]:
!kubectl apply -f ../deployments/test-pod.yaml

pod/k3s-test-pod created


In [13]:
!kubectl get pod k3s-test-pod

NAME           READY   STATUS    RESTARTS   AGE
k3s-test-pod   1/1     Running   0          16s


In [14]:
!kubectl logs pod k3s-test-pod

error: error from server (NotFound): pods "pod" not found in namespace "default"


### K9s (Optional)

If using K9s, link the K3s config so it works out of the box

```bash
    ln -s /etc/rancher/k3s/k3s.yaml ~/.kube/config
```


In [15]:
!ln -s /etc/rancher/k3s/k3s.yaml ~/.kube/config

![K9s pods](../outputs/screenshots/01_k9s-pods.png)

![K9s pulses](../outputs/screenshots/01_k9s-pulses.png)

![K9s xray](../outputs/screenshots/01_k9s-xray-pods.png)

## Summary
- K3s installed and tested

## Cleanup

In [16]:
!kubectl delete pod k3s-test-pod

pod "k3s-test-pod" deleted
^C
